# Ensembl genes table extraction EDA

This notebook is useful for development as well as exploratory data analysis on the extracted tables.
It is currently automically executed and saved as part of exports using `papermill`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ensembl_genes import ensembl_genes
from bioregistry import normalize_prefix

In [3]:
# parameters cell
species = "human"
release = "111"

In [4]:
# Parameters
species = "mouse"
release = "111"


In [5]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release, species=species)
ensg.connection_url

'mysql+mysqlconnector://anonymous@ensembldb.ensembl.org:3306/mus_musculus_core_111_39'

In [6]:
database = ensg.database
database

'mus_musculus_core_111_39'

## Extract data

## gene attrib counts

In [7]:
ensg.run_query("gene_attrib_counts").head(15)

,attrib_type_id,code,name,description,attrib_type_count,attrib_type_examples
0,142,GeneGC,Gene GC,Percentage GC content for this gene,57180,"39.60, 63.29, 36.24, 42.56, 47.54, 47.78, 45.2..."
1,4,name,Name,Alternative/long name,50953,"Arrdc5, Pik3r3, RP24-439I22.9, Ticam1, RP23-30..."
2,395,xref_id,Xref ID,ID of associated database reference,48945,"OTTMUSG00000026352, OTTMUSG00000072373, OTTMUS..."
3,538,legacy_biotype,Legacy biotype,Obsolete biotype previously assigned to this E...,11898,"processed_transcript, lincRNA, antisense, sens..."
4,380,havana_cv,Havana CV term,Controlled vocabulary terms from Havana,6204,"overlapping locus, overlaps pseudogene, readth..."
5,54,remark,Remark,Annotation remark,2475,"TAGENE_gene, confirm experimentally, Transcrib..."
6,382,NoTransRefError,No translations due to reference error,This gene is believed to include protein codin...,5,1
7,1,embl_acc,European Nucleotide Archive (was EMBL) accession,ENA,0,None
8,2,status,Status,None,0,None
9,3,synonym,Synonym,None,0,None


## genes

In [8]:
ensg.gene_df.head()

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
0,ENSMUSG00000000001,5,Gnai3,MGI,MGI:95773,protein_coding,ensembl_havana,2004-11-18 15:05:57,2020-08-10 14:30:16,GRCm39,...,108014596,108053462,-1,True,NaN,None,guanine nucleotide binding protein (G protein)...,MGI Symbol,MGI:95773,ENSMUSG00000000001
1,ENSMUSG00000000003,16,Pbsn,MGI,MGI:1860484,protein_coding,ensembl_havana,2006-06-28 13:29:12,2020-08-10 14:24:33,GRCm39,...,76881507,76897229,-1,True,NaN,None,probasin,MGI Symbol,MGI:1860484,ENSMUSG00000000003
2,ENSMUSG00000000028,16,Cdc45,MGI,MGI:1338073,protein_coding,ensembl_havana,2007-06-26 09:54:29,2020-08-10 14:17:14,GRCm39,...,18599197,18630737,-1,True,NaN,None,cell division cycle 45,MGI Symbol,MGI:1338073,ENSMUSG00000000028
3,ENSMUSG00000000031,19,H19,MGI,MGI:95891,lncRNA,ensembl_havana,2006-02-02 11:27:21,2023-01-03 21:13:04,GRCm39,...,142129262,142131917,-1,True,NaN,None,"H19, imprinted maternally expressed transcript",MGI Symbol,MGI:95891,ENSMUSG00000000031
4,ENSMUSG00000000037,18,Scml2,MGI,MGI:1340042,protein_coding,ensembl_havana,2006-08-18 16:16:32,2020-08-10 14:22:47,GRCm39,...,159865521,160041209,1,True,NaN,None,Scm polycomb group protein like 2,MGI Symbol,MGI:1340042,ENSMUSG00000000037


In [9]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
20633,ENSMUSG00000074720,2,ENSMUSG00000074720,None,None,protein_coding,ensembl,2020-04-27 16:31:13,2020-08-25 18:37:19,GRCm39,...,25923,27230,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000074720
22235,ENSMUSG00000079190,4,ENSMUSG00000079190,None,None,protein_coding,ensembl,2020-04-27 16:17:40,2020-08-25 18:37:25,GRCm39,...,167446,196478,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079190


In [10]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

gene_symbol_source_db,EntrezGene,HGNC,MGI,missing (clone-based),All
ensembl_source,,,,,
RefSeq,0,0,37,0,37
ensembl,3,0,4925,51,4979
ensembl_havana,46,0,22451,35,22532
havana,110,3,26553,439,27105
havana_tagene,0,0,1340,0,1340
mirbase,0,0,1187,0,1187
All,159,3,56493,525,57180


In [11]:
ensg.gene_df.coord_system.value_counts().head(10)

chromosome    57126
scaffold         54
Name: coord_system, dtype: int64

In [12]:
ensg.gene_df.gene_biotype.value_counts().head(10)

protein_coding            21691
lncRNA                    11898
processed_pseudogene      10002
TEC                        3233
unprocessed_pseudogene     2841
miRNA                      2206
snoRNA                     1507
snRNA                      1381
misc_RNA                    562
rRNA                        354
Name: gene_biotype, dtype: int64

In [13]:
pd.crosstab(ensg.gene_df.coord_system, ensg.gene_df.primary_assembly, margins=True)

primary_assembly,False,True,All
coord_system,,,
chromosome,0,57126,57126
scaffold,54,0,54
All,54,57126,57180


In [14]:
ensg.gene_df.mhc.value_counts()

Series([], Name: mhc, dtype: int64)

In [15]:
len(ensg.gene_df)

57180

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [16]:
ensg.alt_allele_df.head()

,ensembl_gene_id,alt_allele_group_id,alt_allele_is_representative,primary_assembly,seq_region,alt_allele_attrib,ensembl_created_date,ensembl_representative_gene_id,is_representative_gene,representative_gene_method


In [17]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

,gene_id,attrib_type_id,value,attrib_type_id,code,name,description


In [18]:
ensg.alt_allele_df.alt_allele_attrib.value_counts()

Series([], Name: alt_allele_attrib, dtype: int64)

In [19]:
ensg.alt_allele_df.query("is_representative_gene").representative_gene_method.value_counts()

Series([], Name: representative_gene_method, dtype: int64)

In [20]:
ensg.gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id


# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [21]:
ensg.old_to_new_df.head(2)

,old_ensembl_gene_id,new_ensembl_gene_id
0,ENSMUSG00000037155,ENSMUSG00000044465
1,ENSMUSG00000000700,ENSMUSG00000066358


In [22]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

ENSMUSG00000070230    131
ENSMUSG00000070062    131
Name: old_ensembl_gene_id, dtype: int64

In [23]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

{'ENSG00000152006'}

In [24]:
ensg.old_to_newest_df.head(2)

,old_ensembl_gene_id,newest_ensembl_gene_id,is_current
0,ENSMUSG00000000700,ENSMUSG00000066358,False
1,ENSMUSG00000000700,ENSMUSG00000068941,False


In [25]:
len(ensg.old_to_newest_df)

71298

In [26]:
ensg.old_to_newest_df.is_current.value_counts()

True     58455
False    12843
Name: is_current, dtype: int64

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [27]:
ensg.update_df.head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
0,ENSMUSG00000000001,ENSMUSG00000000001,True,True,1,1
1,ENSMUSG00000000003,ENSMUSG00000000003,True,True,1,1


In [28]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
34633,ENSMUSG00000075782,ENSMUSG00000119446,False,True,126,254
29462,ENSMUSG00000075909,ENSMUSG00000119294,False,True,126,255


In [29]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
36857,ENSMUSG00000096090,ENSMUSG00000119576,False,True,104,255
37149,ENSMUSG00000075730,ENSMUSG00000119606,False,True,125,255


In [30]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

0.5022268344359406

In [31]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


In [32]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

The omni-updater contains 115,635 rows for mapping 60,984 input genes to 57,180 current, representative genes.


In [33]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


## cross-refrences (xrefs)

In [34]:
ensg.xref_df.head()

,ensembl_representative_gene_id,ensembl_gene_id,gene_symbol,xref_source,xref_accession,xref_label,xref_description,xref_info_type,xref_linkage_annotation,xref_curie
0,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,ArrayExpress,ENSMUSG00000000001,ENSMUSG00000000001,None,DIRECT,None,arrayexpress:ENSMUSG00000000001
1,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,EntrezGene,14679,Gnai3,guanine nucleotide binding protein (G protein)...,DEPENDENT,None,ncbigene:14679
2,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,MGI,MGI:95773,Gnai3,guanine nucleotide binding protein (G protein)...,DIRECT,None,mgi:95773
3,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,Reactome_gene,R-MMU-109582,R-MMU-109582,Hemostasis,DIRECT,None,reactome:R-MMU-109582
4,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,Reactome_gene,R-MMU-111885,R-MMU-111885,Opioid Signalling,DIRECT,None,reactome:R-MMU-111885


In [35]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct 
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

Series([], Name: xref_source, dtype: int64)

In [36]:
# xref sources versus info_types
df = pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)
df["bioregistry_prefix"] = df.index.to_series().replace(ensg._xref_prefix_updater).map(normalize_prefix)
df

xref_info_type,DEPENDENT,DIRECT,MISC,PROJECTION,All,bioregistry_prefix
xref_source,,,,,,
ArrayExpress,0,57180,0,0,57180,arrayexpress
EntrezGene,27786,0,0,0,27786,ncbigene
HGNC,0,0,0,3,3,hgnc
MGI,0,56496,0,0,56496,mgi
RFAM,0,2468,0,0,2468,rfam
Reactome_gene,0,87605,0,0,87605,reactome
Uniprot_gn,51284,0,0,0,51284,uniprot
WikiGene,27786,0,0,0,27786,wikigenes
miRBase,0,0,1360,0,1360,mirbase


## Gene Ontology xrefs

In [37]:
ensg.xref_go_df.head(3)

,ensembl_gene_id,go_id,go_label,go_evidence_codes,xref_info_types,xref_info_texts,ensembl_transcript_ids,ensembl_representative_gene_id
0,ENSMUSG00000000001,GO:0000139,Golgi membrane,"IEA,ISO","PROJECTION,DIRECT","from homo_sapiens translation ENSP00000358867,MGI",ENSMUST00000000001,ENSMUSG00000000001
1,ENSMUSG00000000001,GO:0000166,nucleotide binding,IEA,DIRECT,UniProt,ENSMUST00000000001,ENSMUSG00000000001
2,ENSMUSG00000000001,GO:0001664,G protein-coupled receptor binding,IBA,DIRECT,GO_Central,ENSMUST00000000001,ENSMUSG00000000001


In [38]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label)

[]

## lrg xrefs

In [39]:
ensg.xref_lrg_df.head(2)

,ensembl_gene_id,lrg_gene_id


In [40]:
len(ensg.xref_lrg_df)

0

### ncbigene xrefs

In [41]:
ensg.xref_ncbigene_df.head()

,ensembl_representative_gene_id,ncbigene_id,gene_symbol,ncbigene_symbol
1,ENSMUSG00000000001,14679,Gnai3,Gnai3
26,ENSMUSG00000000003,54192,Pbsn,Pbsn
33,ENSMUSG00000000028,12544,Cdc45,Cdc45
51,ENSMUSG00000000031,14955,H19,H19
55,ENSMUSG00000000037,107815,Scml2,Scml2


In [42]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

ENSMUSG00000000001    1
ENSMUSG00000068962    1
ENSMUSG00000068957    1
Name: ensembl_representative_gene_id, dtype: int64

In [43]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

(27421, 0)

In [44]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

108168560    7
102635519    4
677884       3
Name: ncbigene_id, dtype: int64

In [45]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

(27421, 40)

In [46]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [47]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

57180

In [48]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
17511,ENSMUSG00000062783,3,Csprs,EntrezGene,114564,protein_coding,ensembl,2020-04-27 16:28:04,2020-08-25 18:37:28,GRCm39,...,111572,163011,-1,False,NaN,None,component of Sp100-rs,NCBI gene (formerly Entrezgene),114564,ENSMUSG00000062783
20633,ENSMUSG00000074720,2,ENSMUSG00000074720,None,None,protein_coding,ensembl,2020-04-27 16:31:13,2020-08-25 18:37:19,GRCm39,...,25923,27230,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000074720
22235,ENSMUSG00000079190,4,ENSMUSG00000079190,None,None,protein_coding,ensembl,2020-04-27 16:17:40,2020-08-25 18:37:25,GRCm39,...,167446,196478,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079190
22236,ENSMUSG00000079192,3,ENSMUSG00000079192,None,None,protein_coding,ensembl,2020-04-27 16:11:51,2020-08-25 18:37:27,GRCm39,...,123792,124928,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079192
22239,ENSMUSG00000079222,3,ENSMUSG00000079222,None,None,protein_coding,ensembl,2020-04-27 16:26:11,2020-08-25 18:37:27,GRCm39,...,17591,30203,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079222
22462,ENSMUSG00000079794,3,ENSMUSG00000079794,None,None,protein_coding,ensembl,2020-04-27 16:13:32,2020-08-25 18:37:27,GRCm39,...,135395,136519,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079794
22463,ENSMUSG00000079800,3,ENSMUSG00000079800,None,None,protein_coding,ensembl,2020-04-27 16:09:17,2020-08-25 18:37:26,GRCm39,...,9124,58882,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079800
22465,ENSMUSG00000079808,4,ENSMUSG00000079808,None,None,protein_coding,ensembl,2020-04-27 16:05:55,2020-08-25 18:37:18,GRCm39,...,128555,150452,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079808
30137,ENSMUSG00000091585,3,ENSMUSG00000091585,None,None,protein_coding,ensembl,2020-04-27 17:39:52,2020-08-25 18:37:22,GRCm39,...,716296,720517,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000091585
31305,ENSMUSG00000094054,2,ENSMUSG00000094054,None,None,processed_pseudogene,ensembl,2020-04-27 16:15:58,2020-08-25 18:37:25,GRCm39,...,113868,114758,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000094054
